# Fake news detection

In [1]:
import re
import glob
import numpy as np
import os
import json
import argparse
import time
import codecs
import string
import codecs
import random
import scipy.sparse as sp

from random import randrange
from scipy.sparse import csr_matrix, csc_matrix, hstack, coo_matrix
from gensim.matutils import Scipy2Corpus, corpus2csc
from gensim.models.logentropy_model import LogEntropyModel
from collections import defaultdict, Counter
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, make_scorer,accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords as sw
from string import punctuation
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

E:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Feature extraction functions

In [2]:
#Extracts word-ngrams, when n=1 is equal to bag of words
def wordNgrams(text, n):
    ngrams = []
    text = [word for word in text.split() if word not in string.punctuation]
    ngrams = [' '.join(text[i:i+n])+'' for i in range(len(text)-n+1)]
    return ngrams

In [3]:
txxt = "عمران خان سعودی عرب میں سرمایہ کاری کے حوالے سے کشکول لیے مالی معاونت کی غرض سے ریاض میں موجود ہیں عمران خان کو خود تو اپنی عزت کا احساس نہیں مگر انھیں غیور پاکستانی قوم کی غیرت کا بھی زرا خیال نہ آیا اور سوچے سمجھے بغیر وہ سعودی عرب روانہ ہو گئے جس پر سعودی حکام کو ترس آگیا اور انھوں نے پاکستان کو نہ صرف ایک سال کے لیے 10 ارب ڈالر دینے کا اعلان کیا بلکہ 5 سال تک مؤخر ادائیگیوں پر تقریباً 15 ارب ڈالر مالیت تک کا تیل دینے پر بھی رضامند ہو گیا ہے  دفترِ خارجہ کا کہنا ہے کہ سعودی عرب کی جانب سے 10 ارب ڈالر دیے جانے والی مالی امداد سے بیرون ملکی قرضوں کی ادائیگیوں میں توازن لانے میں مدد دیں گے وزیراعظم ہاؤس اعلامیے کے مطابق سعودی عرب نے پاکستان میں نہ صرف آئل ریفائنری لگانے کا فیصلہ کیا ہے بلکہ بلوچستان میں معدنیات کی کانوں میں بھی سے معدنیات بھی نکالنے کے لیے کثر رقم فراہم کرے گا اور اس سلسلے میں ان کی حکومت کے 20 ممبران پر مشتمل اعلیٰ وفد کو وزیراعظم عمران خان نے پاکستان کے دورے پر آنے کی دعوت دی ہے  وزیراعظم عمران خان نے دورہ سعودی عرب میں ان کے ہمراہ وزیر خارجہ شاہ محمود قریشی وزیر خزانہ شیخ رشید وزیر اطلاعات معاز احمد اور وزیراعظم کے مشیر برائے تجارت آصف علی زرداری بھی شامل تھے پاکستان کے معاشی بحران سے نمٹنے کے لیے عمران خان نے پہلے ہی کہا تھا کہ وہ آئی ایم ایف سے مزید قرض نہیں لینا چاہتے کیونکہ وہ ہم واپس کرنے کی صلاحیت نہیں رکھے گے اس لیے بہتر ہو گا کہ اسلامی دوست ممالک سے مالی امداد کرنے کی التجا کرنے کا دروازہ کو ترجیح دیں گے اس لیے کشکول لیے مالی معاونت کی غرض سے عمران خان نے وزیراعظم کا بنتے کہ سعودی عرب کا ہی انتخاب کیا تھا کیونکہ مانا جاتا ہے کہ عربوں سے پاس بہت پیسہ اور تیک لے وسیع و عریض زخائر ہیں دورے کے بعد معاز احمد نے اسلام آباد میں پریس کانفرنس سے خطاب کرتے ہوئے کہا تھا کہ سعودی عرب وہ پہلا ملک ہے جسے پاکستان نے پاکستان کی مالی معاونت کے لیے اپنے خزانوں کے منہ کھول دہے ہیں وہ سعودی عرب کے ایک اعلیٰ سطح کے وفد کا خود انتظار کریں گے کہ وہ پاکستان کا دورہ کریں اور سرمایہ کاری کے منصوبوں کے لیے  مزید امدادی پیکج کا واضح اعلان کریں تاکہ بیرون ملکی قرضوں کی ادائیگیوں میں توازن لایا جا سکے "
wordNgrams(txxt,2)

['عمران خان',
 'خان سعودی',
 'سعودی عرب',
 'عرب میں',
 'میں سرمایہ',
 'سرمایہ کاری',
 'کاری کے',
 'کے حوالے',
 'حوالے سے',
 'سے کشکول',
 'کشکول لیے',
 'لیے مالی',
 'مالی معاونت',
 'معاونت کی',
 'کی غرض',
 'غرض سے',
 'سے ریاض',
 'ریاض میں',
 'میں موجود',
 'موجود ہیں',
 'ہیں عمران',
 'عمران خان',
 'خان کو',
 'کو خود',
 'خود تو',
 'تو اپنی',
 'اپنی عزت',
 'عزت کا',
 'کا احساس',
 'احساس نہیں',
 'نہیں مگر',
 'مگر انھیں',
 'انھیں غیور',
 'غیور پاکستانی',
 'پاکستانی قوم',
 'قوم کی',
 'کی غیرت',
 'غیرت کا',
 'کا بھی',
 'بھی زرا',
 'زرا خیال',
 'خیال نہ',
 'نہ آیا',
 'آیا اور',
 'اور سوچے',
 'سوچے سمجھے',
 'سمجھے بغیر',
 'بغیر وہ',
 'وہ سعودی',
 'سعودی عرب',
 'عرب روانہ',
 'روانہ ہو',
 'ہو گئے',
 'گئے جس',
 'جس پر',
 'پر سعودی',
 'سعودی حکام',
 'حکام کو',
 'کو ترس',
 'ترس آگیا',
 'آگیا اور',
 'اور انھوں',
 'انھوں نے',
 'نے پاکستان',
 'پاکستان کو',
 'کو نہ',
 'نہ صرف',
 'صرف ایک',
 'ایک سال',
 'سال کے',
 'کے لیے',
 'لیے 10',
 '10 ارب',
 'ارب ڈالر',
 'ڈالر دینے',
 'دینے کا',
 'کا اعلان',
 'اعلان 

In [4]:
#Extracts character n-grams
def charNgrams(text, n):
    ngrams = []
    ngrams = [text[i:i+n]+'_cng' for i in range(len(text)-n+1)]
    return ngrams

In [5]:
charNgrams(txxt,2)

['عم_cng',
 'مر_cng',
 'را_cng',
 'ان_cng',
 'ن _cng',
 ' خ_cng',
 'خا_cng',
 'ان_cng',
 'ن _cng',
 ' س_cng',
 'سع_cng',
 'عو_cng',
 'ود_cng',
 'دی_cng',
 'ی _cng',
 ' ع_cng',
 'عر_cng',
 'رب_cng',
 'ب _cng',
 ' م_cng',
 'می_cng',
 'یں_cng',
 'ں _cng',
 ' س_cng',
 'سر_cng',
 'رم_cng',
 'ما_cng',
 'ای_cng',
 'یہ_cng',
 'ہ _cng',
 ' ک_cng',
 'کا_cng',
 'ار_cng',
 'ری_cng',
 'ی _cng',
 ' ک_cng',
 'کے_cng',
 'ے _cng',
 ' ح_cng',
 'حو_cng',
 'وا_cng',
 'ال_cng',
 'لے_cng',
 'ے _cng',
 ' س_cng',
 'سے_cng',
 'ے _cng',
 ' ک_cng',
 'کش_cng',
 'شک_cng',
 'کو_cng',
 'ول_cng',
 'ل _cng',
 ' ل_cng',
 'لی_cng',
 'یے_cng',
 'ے _cng',
 ' م_cng',
 'ما_cng',
 'ال_cng',
 'لی_cng',
 'ی _cng',
 ' م_cng',
 'مع_cng',
 'عا_cng',
 'او_cng',
 'ون_cng',
 'نت_cng',
 'ت _cng',
 ' ک_cng',
 'کی_cng',
 'ی _cng',
 ' غ_cng',
 'غر_cng',
 'رض_cng',
 'ض _cng',
 ' س_cng',
 'سے_cng',
 'ے _cng',
 ' ر_cng',
 'ری_cng',
 'یا_cng',
 'اض_cng',
 'ض _cng',
 ' م_cng',
 'می_cng',
 'یں_cng',
 'ں _cng',
 ' م_cng',
 'مو_cng',
 'وج_cng',

In [6]:
def load_diccionario(ruta):
    terms = set()#Dictionary of slangs
    try:
        tmp = open(ruta, "r", encoding="utf8" )     
        while True :
            linea = tmp.readline()                                                                                   
            #linea = to_unicode(linea) 
            if (not linea) or (linea == ""):                                                                               
                break;                                                                                                      
            linea = linea.rstrip()
            terms.add(linea.lower())
        return (terms)
    except IOError as e:
        print ("Error: "+ruta+" I/O error({0}): {1}".format(e.errno, e.strerror))
        exit(1)

In [7]:
#Extracts function words n-grams with a pre-loaded dictionary
def funcNgrams(text, n):
    stop_words = load_diccionario('stop_words.txt')
    patt=r'\b(' + ('|'.join(re.escape(key) for key in stop_words)).lstrip('|') + r')\b'
    pattern = re.compile(patt)
    text = re.sub(r"(\n+|\r+|(\r\n)+)", " ", text)
    text = re.sub(r" +", " ", text)
    text = re.sub(r"’", "'", text)
    text = re.sub(r"[" + punctuation + "]*", "", text)
    terms = pattern.findall(text)
    n_grams=[('_'.join(terms[i:i+n])) + "_fwn" for i in range(len(terms)-n+1)]

    return n_grams

In [8]:
print(funcNgrams(txxt,1))
len(funcNgrams(txxt,1))

['میں_fwn', 'کے_fwn', 'سے_fwn', 'لیے_fwn', 'کی_fwn', 'سے_fwn', 'میں_fwn', 'ہیں_fwn', 'کو_fwn', 'خود_fwn', 'تو_fwn', 'اپنی_fwn', 'کا_fwn', 'نہیں_fwn', 'مگر_fwn', 'کی_fwn', 'کا_fwn', 'بھی_fwn', 'نہ_fwn', 'آیا_fwn', 'اور_fwn', 'بغیر_fwn', 'وہ_fwn', 'ہو_fwn', 'گئے_fwn', 'جس_fwn', 'پر_fwn', 'کو_fwn', 'اور_fwn', 'نے_fwn', 'کو_fwn', 'نہ_fwn', 'صرف_fwn', 'ایک_fwn', 'کے_fwn', 'لیے_fwn', 'دینے_fwn', 'کا_fwn', 'کیا_fwn', 'بلکہ_fwn', 'تک_fwn', 'پر_fwn', 'تک_fwn', 'کا_fwn', 'دینے_fwn', 'پر_fwn', 'بھی_fwn', 'ہو_fwn', 'گیا_fwn', 'ہے_fwn', 'کا_fwn', 'ہے_fwn', 'کہ_fwn', 'کی_fwn', 'سے_fwn', 'دیے_fwn', 'جانے_fwn', 'والی_fwn', 'سے_fwn', 'کی_fwn', 'میں_fwn', 'لانے_fwn', 'میں_fwn', 'دیں_fwn', 'گے_fwn', 'کے_fwn', 'نے_fwn', 'میں_fwn', 'نہ_fwn', 'صرف_fwn', 'کا_fwn', 'کیا_fwn', 'ہے_fwn', 'بلکہ_fwn', 'میں_fwn', 'کی_fwn', 'میں_fwn', 'بھی_fwn', 'سے_fwn', 'بھی_fwn', 'کے_fwn', 'لیے_fwn', 'کرے_fwn', 'گا_fwn', 'اور_fwn', 'اس_fwn', 'میں_fwn', 'ان_fwn', 'کی_fwn', 'کے_fwn', 'پر_fwn', 'کو_fwn', 'نے_fwn', 'کے_fwn', 'پر_fwn

210

In [9]:
def extract_features(text,cn,wn,fn):
    text = text.lower()
    #text=clean_text(text)
    features = []
    for n in wn:
        if n != 0:
            features.extend(wordNgrams(text,n))
    for n in cn:
        if n != 0:
            features.extend(charNgrams(text,n))
    for n in fn:
            if n != 0:
                features.extend(funcNgrams(text,n))
    return features

In [10]:
# Extracts all features in a set of 'texts' and return as a string separated with the simbol '&%$'
def process_texts(texts,cn,wn,fn):
    occurrences=defaultdict(int)
    featuresList=[]
    featuresDict=Counter()
    for (text) in texts:
        features=extract_features(text,cn,wn,fn)
        featuresDict.update(features)
        featuresList.append('&%$'.join(features))
    return featuresList, featuresDict

## Reading and preparing the corpus

In [11]:
def preprocessText(text):
    cleantext = re.sub('\d','0',text)
    return cleantext

When I do want to remove stop words. I should use this (below) function 1. Anyhow I need to use either function 1 or function 2. I can't use both at the same time. I mean I need to comment one of the functions

In [12]:
# # Function 1
# # utility function for reading files
# def read_txt_files(files):
#     text=[]
#     topic=[]
#     for i,file_path in sorted(enumerate(files)):
#         print('read', file_path)
#         with open(file_path,"r", encoding="utf8") as infile:
#             cleantext = preprocessText(infile.read())
#             text.append(cleantext)
#             file_topic=''.join(re.findall('[A-Za-z]',file_path.split('\\')[3].split('.')[0]))
#             topic.append(file_topic)
#     return text, topic

When I do not want to remove stop words. I should use this (below) function 2

In [13]:
#Function 2
#utility function for reading files
def read_txt_files(files):
    text=[]
    topic=[]
    for i,file_path in sorted(enumerate(files)):
        print('read', file_path)
        with open(file_path,"r", encoding="utf8") as infile:
            text.append(infile.read())
            file_topic=''.join(re.findall('[A-Za-z]',file_path.split('\\')[3].split('.')[0]))
            topic.append(file_topic)
    return text, topic

In [14]:
#reading the path of real and fake news for training
train_path_real='Corpus\\Train\\Real\\'
train_path_fake='Corpus\\Train\\Fake\\'

real_news, real_news_topics = read_txt_files(sorted(glob.glob(train_path_real+'*.txt')))

fake_news, fake_news_topics = read_txt_files(sorted(glob.glob(train_path_fake+'*.txt')))

#contatenating real and fake news in one variable for training
train_texts = np.concatenate((real_news, fake_news))
train_labels = np.concatenate((np.ones(len(real_news)), np.zeros(len(fake_news))))
train_topics = np.concatenate((real_news_topics, fake_news_topics))

read Corpus\Train\Real\bus1.txt
read Corpus\Train\Real\bus10.txt
read Corpus\Train\Real\bus11.txt
read Corpus\Train\Real\bus12.txt
read Corpus\Train\Real\bus13.txt
read Corpus\Train\Real\bus14.txt
read Corpus\Train\Real\bus15.txt
read Corpus\Train\Real\bus16.txt
read Corpus\Train\Real\bus17.txt
read Corpus\Train\Real\bus18.txt
read Corpus\Train\Real\bus19.txt
read Corpus\Train\Real\bus2.txt
read Corpus\Train\Real\bus20.txt
read Corpus\Train\Real\bus21.txt
read Corpus\Train\Real\bus22.txt
read Corpus\Train\Real\bus23.txt
read Corpus\Train\Real\bus24.txt
read Corpus\Train\Real\bus25.txt
read Corpus\Train\Real\bus26.txt
read Corpus\Train\Real\bus27.txt
read Corpus\Train\Real\bus28.txt
read Corpus\Train\Real\bus29.txt
read Corpus\Train\Real\bus3.txt
read Corpus\Train\Real\bus30.txt
read Corpus\Train\Real\bus31.txt
read Corpus\Train\Real\bus32.txt
read Corpus\Train\Real\bus33.txt
read Corpus\Train\Real\bus34.txt
read Corpus\Train\Real\bus35.txt
read Corpus\Train\Real\bus36.txt
read Corpus\T

read Corpus\Train\Fake\hlth7.txt
read Corpus\Train\Fake\hlth70.txt
read Corpus\Train\Fake\hlth8.txt
read Corpus\Train\Fake\hlth9.txt
read Corpus\Train\Fake\sbz1.txt
read Corpus\Train\Fake\sbz10.txt
read Corpus\Train\Fake\sbz11.txt
read Corpus\Train\Fake\sbz12.txt
read Corpus\Train\Fake\sbz13.txt
read Corpus\Train\Fake\sbz14.txt
read Corpus\Train\Fake\sbz15.txt
read Corpus\Train\Fake\sbz16.txt
read Corpus\Train\Fake\sbz17.txt
read Corpus\Train\Fake\sbz18.txt
read Corpus\Train\Fake\sbz19.txt
read Corpus\Train\Fake\sbz2.txt
read Corpus\Train\Fake\sbz20.txt
read Corpus\Train\Fake\sbz21.txt
read Corpus\Train\Fake\sbz22.txt
read Corpus\Train\Fake\sbz23.txt
read Corpus\Train\Fake\sbz24.txt
read Corpus\Train\Fake\sbz25.txt
read Corpus\Train\Fake\sbz26.txt
read Corpus\Train\Fake\sbz27.txt
read Corpus\Train\Fake\sbz28.txt
read Corpus\Train\Fake\sbz29.txt
read Corpus\Train\Fake\sbz3.txt
read Corpus\Train\Fake\sbz30.txt
read Corpus\Train\Fake\sbz31.txt
read Corpus\Train\Fake\sbz32.txt
read Corpus\

In [15]:
print ('Train:')
print ('\t Real:',len(real_news))
print ('\t Fake:',len(fake_news))

Train:
	 Real: 350
	 Fake: 288


In [16]:
#reading the path of real and fake news for testing
test_path_real='corpus\\Test\\Real\\'
test_path_fake='corpus\\Test\\Fake\\'

real_news, real_news_topics = read_txt_files(sorted(glob.glob(test_path_real+'*.txt')))
fake_news, fake_news_topics = read_txt_files(sorted(glob.glob(test_path_fake+'*.txt')))

#contatenating real and fake news in one variable for testing
test_texts = np.concatenate((real_news, fake_news))
test_labels = np.concatenate((np.ones(len(real_news)), np.zeros(len(fake_news))))
test_topics = np.concatenate((real_news_topics, fake_news_topics))

read corpus\Test\Real\bus100.txt
read corpus\Test\Real\bus71.txt
read corpus\Test\Real\bus72.txt
read corpus\Test\Real\bus73.txt
read corpus\Test\Real\bus74.txt
read corpus\Test\Real\bus75.txt
read corpus\Test\Real\bus76.txt
read corpus\Test\Real\bus77.txt
read corpus\Test\Real\bus78.txt
read corpus\Test\Real\bus79.txt
read corpus\Test\Real\bus80.txt
read corpus\Test\Real\bus81.txt
read corpus\Test\Real\bus82.txt
read corpus\Test\Real\bus83.txt
read corpus\Test\Real\bus84.txt
read corpus\Test\Real\bus85.txt
read corpus\Test\Real\bus86.txt
read corpus\Test\Real\bus87.txt
read corpus\Test\Real\bus88.txt
read corpus\Test\Real\bus89.txt
read corpus\Test\Real\bus90.txt
read corpus\Test\Real\bus91.txt
read corpus\Test\Real\bus92.txt
read corpus\Test\Real\bus93.txt
read corpus\Test\Real\bus94.txt
read corpus\Test\Real\bus95.txt
read corpus\Test\Real\bus96.txt
read corpus\Test\Real\bus97.txt
read corpus\Test\Real\bus98.txt
read corpus\Test\Real\bus99.txt
read corpus\Test\Real\hlth100.txt
read 

In [24]:
print ('Test:')
print ('\t Real:',len(real_news))
print ('\t Fake:',len(fake_news))

Test:
	 Real: 150
	 Fake: 112


## Parametrization and feature extraction

In [245]:
# Parameters
cnvalues=[0] #character n-grams
wnvalues=[1] # word n-grams; bag of words
fnvalues=[0] # function words n-grams

In [246]:
#Train feature extraction
print('Extracting features')
train_features, dicOfFeatures = process_texts(train_texts,cnvalues,wnvalues,fnvalues)

vectorizer = CountVectorizer(lowercase=False, min_df=2, tokenizer=lambda x: x.split('&%$')) #--> we can change this
train_data = vectorizer.fit_transform(train_features)
train_data = train_data.astype(float)
print('\t', 'Labels for each document: ', len(train_labels))
print('\t', 'Total training files (Real + Fake) : ', len(train_texts))
print('\t', 'Vocabulary size of', len(train_texts), 'files is : ',len(dicOfFeatures))
print ('\t','Train shape:',train_data.shape)
print('\t', 'class dictribution',Counter(train_labels))

Extracting features
	 Labels for each document:  638
	 Total training files (Real + Fake) :  638
	 Vocabulary size of 638 files is :  15848
	 Train shape: (638, 8564)
	 class dictribution Counter({1.0: 350, 0.0: 288})


Labels for each documents means that we have total 638 files in which first 350 files are assigned value 1 (we assign value 1 if a documents belongs to real class, and we assigned value zero if a documents belongs to fake class), and rest 288 files are assigned value zero. 

In [247]:
# Test feature extraction
print('Extracting Test features')
test_features,dicOfFeaturesTest = process_texts(test_texts,cnvalues,wnvalues,fnvalues)
test_data = vectorizer.transform(test_features)
test_data = test_data.astype(float)

print('\t', 'Total testing files (Real + Fake): ', len(test_texts))
print('\t', 'vocabulary size: ',len(dicOfFeaturesTest))
print ('\t','Test shape:',test_data.shape)
print('\t', 'class dictribution: ',Counter(test_labels))

Extracting Test features
	 Total testing files (Real + Fake):  262
	 vocabulary size:  10298
	 Test shape: (262, 8564)
	 class dictribution:  Counter({1.0: 150, 0.0: 112})


## Frequency threshold

N = 5 means we remove all the words from the train data that has frequency less than 5.

In [248]:
N=5
X=train_data
values=np.array(X.sum(axis=0)).ravel()
thresholdMask=(values >= N)*1
indices_zero = list(np.nonzero(thresholdMask == 0)[0])
all_cols = np.arange(X.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, indices_zero)))[0]
train_data = X[:, cols_to_keep]
#####

scaled_train_data=train_data
scaled_test_data=test_data
print('Train shape:',scaled_train_data.shape)
print('Test shape:',scaled_test_data.shape)

Train shape: (638, 4099)
Test shape: (262, 8564)


638 means the number of deocuments and 4099 means all those words who have word frequency atleast 5. 

To perfrom the experiments, the dimensions of testing and training features should
be the same. In order to see the testing features, the below cell will perform this 
task.

In [249]:
Z=test_data
all_cols = np.arange(Z.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, indices_zero)))[0]
test_data = Z[:, cols_to_keep]
scaled_test_data=test_data
print('Test shape:',scaled_test_data.shape)

Test shape: (262, 4099)


## Weighting schemes

In [250]:
# print ('only frecuency:',test_data)
feature_weight='binary' # possible values: binary, logent, tfidf, norm, relat

if feature_weight == 'binary':
    scaled_train_data = preprocessing.Binarizer().fit_transform(scaled_train_data)
    scaled_test_data = preprocessing.Binarizer().fit_transform(scaled_test_data)
    print ("feature_weight = binary")
    
elif feature_weight == 'logent':
    Xc = Scipy2Corpus(scaled_train_data)
    log_ent = LogEntropyModel(Xc)
    X = log_ent[Xc]
    X = corpus2csc(X)
    scaled_train_data = sp.csc_matrix.transpose(X)
    
    Xc = Scipy2Corpus(scaled_test_data)
    log_ent = LogEntropyModel(Xc)
    X = log_ent[Xc]
    X = corpus2csc(X)
    scaled_test_data = sp.csc_matrix.transpose(X)
    print ("feature_weight = logent")
    
elif feature_weight == 'tfidf':
    transformer = TfidfTransformer()
    scaled_train_data = transformer.fit_transform(scaled_train_data)
    scaled_test_data = transformer.transform(scaled_test_data)
    print ("feature_weight = tfidf")
    
elif feature_weight=='norm':
    scaled_train_data = preprocessing.normalize(scaled_train_data, norm='l2')
    max_abs_scaler = preprocessing.MaxAbsScaler()
    scaled_train_data = max_abs_scaler.fit_transform(scaled_train_data)
    scaled_test_data = max_abs_scaler.transform(scaled_test_data)
    print ("feature_weight = norm")
    
elif feature_weight=='relat':
    s = scaled_train_data.sum(axis = 1)
    scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/s))
    s = scaled_test_data.sum(axis = 1)
    scaled_test_data = coo_matrix(np.nan_to_num(scaled_test_data/s))
    print ("feature_weight = relat")
    
else:
    print ("feature_weight = tf")
    
# print ('with weighting scheme:',scaled_test_data)

feature_weight = binary


## Classification Process - Training

In [251]:
#Utility function
originalclass=[]
predictedclass=[]
def classification_report_with_f1_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return f1_score(y_true, y_pred) # return accuracy score

In [252]:
print('Training Classifier')
    
# Applying classification algorithms
clf=LinearSVC(C=0.01,class_weight='balanced', random_state=85)
clfSVC=SVC(C=0.01, kernel='linear',class_weight='balanced')
clfSVC.fit(scaled_train_data, train_labels)
clfMnb=MultinomialNB()
clfMnb.fit(scaled_train_data, train_labels)
clfBnb=BernoulliNB()
clfBnb.fit(scaled_train_data, train_labels)
clfLG=LogisticRegression(solver='lbfgs', tol=0.001, C=0.01,class_weight='balanced')
clfLG.fit(scaled_train_data, train_labels)
clfDT=DecisionTreeClassifier(random_state=0)
clfDT.fit(scaled_train_data, train_labels)
clfRFC=RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clfRFC.fit(scaled_train_data, train_labels)
clfAB=AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0,  random_state=None)
clfAB.fit(scaled_train_data, train_labels)

nested_score = cross_val_score(clf, X=scaled_train_data, y=train_labels, cv=10, scoring=make_scorer(classification_report_with_f1_score))
#cvScoreLinearSVC=cross_val_score(clf, scaled_train_data, train_labels, cv=10, scoring='f1').mean()

cvScoreMnb=cross_val_score(clfMnb, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Multinomial Naive Bayes',cvScoreMnb)

cvScoreBnb=cross_val_score(clfBnb, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Bernoulli Naive Bayes',cvScoreBnb)

cvScoreSVC=cross_val_score(clfSVC, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Linear SVC',cvScoreSVC)

cvScoreLG=cross_val_score(clfLG, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Logistic Regression',cvScoreLG)

cvScoreAD=cross_val_score(clfAB, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation AdaBoost',cvScoreAD)

cvScoreDT=cross_val_score(clfDT, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Decision Tree',cvScoreDT)

cvScoreRFC=cross_val_score(clfRFC, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Random Forest',cvScoreRFC)

# print('10-Fold Cross-validation Linear SVC',nested_score.mean())
# print(classification_report(originalclass, predictedclass))

Training Classifier
10-Fold Cross-validation Multinomial Naive Bayes 0.5356521159838389
10-Fold Cross-validation Bernoulli Naive Bayes 0.47506957885014167
10-Fold Cross-validation Linear SVC 0.6887970505191532
10-Fold Cross-validation Logistic Regression 0.6580015855002737
10-Fold Cross-validation AdaBoost 0.7065458174995973
10-Fold Cross-validation Decision Tree 0.6300598955974196
10-Fold Cross-validation Random Forest 0.7158644815725663


## Classification Process - Testing

#### Testing for Testing Classifiers

In [253]:
#Testing MultinomialNB
predictions=clfMnb.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       0.66      0.75      0.70       112
        1.0       0.79      0.71      0.75       150

avg / total       0.73      0.73      0.73       262

Accuracy 0.7251908396946565
F1-score 0.7464788732394366


In [254]:
#Testing BernoulliNB
predictions=clfBnb.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       0.58      0.84      0.69       112
        1.0       0.82      0.55      0.66       150

avg / total       0.72      0.68      0.67       262

Accuracy 0.6755725190839694
F1-score 0.6613545816733067


In [255]:
#Testing Support vector Machine3
predictions=clfSVC.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       0.74      0.62      0.67       112
        1.0       0.75      0.84      0.79       150

avg / total       0.74      0.74      0.74       262

Accuracy 0.7442748091603053
F1-score 0.7899686520376176


In [256]:
#Testing LogisticRegression
predictions=clfLG.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       0.72      0.61      0.66       112
        1.0       0.74      0.82      0.78       150

avg / total       0.73      0.73      0.73       262

Accuracy 0.7290076335877863
F1-score 0.77602523659306


In [257]:
#Testing Adaboost
predictions=clfAB.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       0.66      0.62      0.64       112
        1.0       0.73      0.76      0.74       150

avg / total       0.70      0.70      0.70       262

Accuracy 0.6984732824427481
F1-score 0.742671009771987


In [258]:
#Testing DecisionTreeClassifier
predictions=clfDT.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       0.57      0.60      0.59       112
        1.0       0.69      0.67      0.68       150

avg / total       0.64      0.64      0.64       262

Accuracy 0.6374045801526718
F1-score 0.6779661016949153


In [259]:
#Testing Random Forest
predictions=clfRFC.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))

             precision    recall  f1-score   support

        0.0       1.00      0.03      0.05       112
        1.0       0.58      1.00      0.73       150

avg / total       0.76      0.58      0.44       262

Accuracy 0.583969465648855
F1-score 0.7334963325183373
